In [2]:
import faiss
import numpy as np

d = 128  

index1 = faiss.IndexFlatL2(d)
index2 = faiss.IndexFlatL2(d)

np.random.seed(42)
data1 = np.random.random((1000, d)).astype('float32') 
data2 = np.random.random((1000, d)).astype('float32')  

index1.add(data1)
index2.add(data2)

print("Index1 size:", index1.ntotal)  
print("Index2 size:", index2.ntotal)  



Index1 size: 1000
Index2 size: 1000


In [9]:
query = np.random.random((2, d)).astype('float32')

In [11]:
dist1, idx1 = index1.search(query, k=10) 
print("Kết quả từ Index1:", idx1)

Kết quả từ Index1: [[231 388 958 969 437 321 129 551 824 197]
 [771  71 241 805 729 293 116 419 662  94]]


In [7]:
dist2, idx2 = index2.search(query, k=10)
print("Kết quả từ Index2:", idx2, dist2)

Kết quả từ Index2: [[182 329 124 288 794 572 808 891 632  76]] [[15.919629 16.045359 16.072968 16.137405 16.324867 16.410614 16.50709
  16.518627 16.727901 16.75441 ]]


In [8]:
type(idx1)

numpy.ndarray

In [12]:
import faiss
import numpy as np

# --- 1. Tạo 2 FAISS index ảo ---
d = 128
index1 = faiss.IndexFlatL2(d)
index2 = faiss.IndexFlatL2(d)

np.random.seed(42)
data1 = np.random.random((10, d)).astype('float32')
data2 = np.random.random((15, d)).astype('float32')

index1.add(data1)
index2.add(data2)

# Query vector
query = np.random.random((1, d)).astype('float32')

# --- 2. Lấy kết quả từ 2 index ---
k = 5
dist1, idx1 = index1.search(query, k)
dist2, idx2 = index2.search(query, k)

# Đổi index sang dạng list cho dễ xử lý
rankings = [
    idx1[0].tolist(),  # ranking list từ DB1
    idx2[0].tolist(),  # ranking list từ DB2
]

# --- 3. Áp dụng Reciprocal Rank Fusion ---
def reciprocal_rank_fusion(rank_lists, k_param=60):
    scores = {}
    for rank_list in rank_lists:
        for rank_pos, doc_id in enumerate(rank_list):
            scores[doc_id] = scores.get(doc_id, 0) + 1 / (k_param + rank_pos + 1)
    return sorted(scores.items(), key=lambda x: x[1], reverse=True)

rrf_results = reciprocal_rank_fusion(rankings)

print("Ranking từ Index1:", rankings[0])
print("Ranking từ Index2:", rankings[1])
print("Kết quả hợp nhất RRF:", rrf_results)


Ranking từ Index1: [5, 8, 6, 9, 0]
Ranking từ Index2: [14, 13, 6, 12, 4]
Kết quả hợp nhất RRF: [(6, 0.031746031746031744), (5, 0.01639344262295082), (14, 0.01639344262295082), (8, 0.016129032258064516), (13, 0.016129032258064516), (9, 0.015625), (12, 0.015625), (0, 0.015384615384615385), (4, 0.015384615384615385)]
